In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Please run this code with the kernel reinvent.v3.2

In [12]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

from utils import fingerprints_from_mol

from scripts.acquisition import select_query

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [13]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]


In [14]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: UNKNOWN
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [15]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [16]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [17]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Successfully uninstalled reinvent-models-0.0.15rc1
  Running setup.py develop for reinvent-models


In [18]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring

In [75]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = f"rounds_{num_rounds}_iters_{num_iters}_queries_{num_queries}_{acquisition}_noise_{sigma_noise}"
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # Copy the pretrained model to the first REINVENT round/ HITL_iteration_1
    shutil.copy2(f"{model_pretrained_path}", f"{output_dir}/REINVENT_round_1/HITL_iteration_1")

    # multi-parameter optimization (MPO) loop
    print(f"Running DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}")
    print(f"Results will be saved at {output_dir}")
    
    base_training_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type, 
                                                 data_path=base_training_dataset_path)
    base_testing_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type,
                                                data_path=base_testing_dataset_path)
    
    print("Loading initial training and testing datasets successfully")

    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")

        REINVENT_round_output_dir = f"{output_dir}/REINVENT_round_{REINVENT_round}"
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                                        REINVENT_round_output_dir, conf_filename)

        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        current_model_path = f"{REINVENT_round_output_dir}/HITL_iteration_1/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)
    
        print("Run REINVENT")                
        command = f"{reinvent_env}/bin/python"
        script = f"{reinvent_dir}/input.py"
        stderr_file = f"{REINVENT_round_output_dir}/run.err"
        stdout_file = f"{REINVENT_round_output_dir}/run.out"

        # Construct the full command to run
        cmd = [command, script, configuration_JSON_path]
        # Open the file to which you want to redirect stderr and stdout
        with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
            # Execute the command
            result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
        # Check the result
        print("Exit code:", result.returncode)
        
        #############################################################################
        # REINVENT HAS OUTPUT THE RESULT in path f"{REINVENT_round_output_dir}/results" #
        #############################################################################
        
        # Get the high scoring molecules
        output_high_score = read_scaffold_result(f"{REINVENT_round_output_dir}/results/scaffold_memory.csv", 
                                                 choose_top_smiles=choose_top_smiles)
        
        scaffold_df = output_high_score["scaffold_df"]
        smiles = output_high_score["smiles"]
        
        # store molecule indexes selected for feedback
        selected_feedback = np.empty(0).astype(int)

        ########################### HITL_iteration in each REINVENT round #####################
        
        for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations

            print("=====================================")
            print(f"HITL iteration = {HITL_iteration}")
            
            # Initialize human feedback model
            print(f"Loading {feedback_type} feedback model.")

            # Loading feedback model
            feedback_model_path = f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"
            feedback_model = load_feedback_model(feedback_type, feedback_model_path)
            
            ######################################################## 
            # Select queries number of smiles with Active Learning #
            ########################################################

            if len(smiles) > num_queries:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, num_queries, list(smiles), 
                                                  selected_feedback, acquisition, rng) 
            else:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, len(smiles), list(smiles), 
                                                  selected_feedback, acquisition, rng)
            
            print(f"Feedback idx at HITL iteration {HITL_iteration}: {new_queried_smiles_indices}")
            
            new_queried_smiles = [smiles[i] for i in new_queried_smiles_indices]
  
            selected_feedback = np.hstack((selected_feedback, new_queried_smiles_indices))

            new_queried_smiles_human_score = smiles_human_score(new_queried_smiles, sigma_noise)
            
            print(f"Human score at HITL iteration {HITL_iteration}: {new_queried_smiles_human_score}")
            
            # use the augmented training data to retrain the model
            new_queried_fps = [compute_fingerprints(smiles) for smiles in new_queried_smiles]

            iteration_training_dataset_outputs = create_drd2_dataset(feedback_type, 
                                                                      new_queried_smiles, 
                                                                      new_queried_smiles_human_score,
                                                                      new_queried_fps)
            
            print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles'])}")

            # combining the base training dataset with the new queried dataset
            base_training_dataset_outputs = combine_drd2_dataset(feedback_type, base_training_dataset_outputs, 
                                                                   iteration_training_dataset_outputs)
            
            print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles'])}")

            # save augmented training data
            save_drd2_dataset(feedback_type, iteration_training_dataset_outputs, f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_queried_data.csv")
            save_drd2_dataset(feedback_type, base_training_dataset_outputs, f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            print(f"Saved augmented training data at {REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            # Retraining the feedback model using the augmented train set
            retrained_feedback_model = retrain_feedback_model(feedback_type, feedback_model, base_training_dataset_outputs)
            
            # At the last round and last iteration, we no longer need to copy the model to the next round
            if REINVENT_round != num_rounds and HITL_iteration != num_iters:
                if HITL_iteration < num_iters:
                    # Moving on to the next iteration at the current round
                    feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration + 1}/{model_pretrained_name}"
                else:
                    # Moving to the first iteration at the next round
                    feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round + 1}/HITL_iteration_1/{model_pretrained_name}"
                
                # save the torch model
                torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)

    #         # get current configuration
    #         configuration = json.load(open(os.path.join(output_dir, conf_filename)))
    #         conf_filename = "HITL_iteration{}_config.json".format(REINVENT_round)    

    #         # modify model path in configuration
    #         configuration_scoring_function = configuration["parameters"]["scoring_function"]["parameters"]
    #         for i in range(len(configuration_scoring_function)):
    #             if configuration_scoring_function[i]["component_type"] == "predictive_property":
    #                 configuration_scoring_function[i]["specific_parameters"]["model_path"] = model_new_savefile

    #         # Keep agent checkpoint
    #         if REINVENT_round == 1:
    #             configuration["parameters"]["reinforcement_learning"]["agent"] = os.path.join(initial_dir, "results/Agent.ckpt")
    #         else:
    #             configuration["parameters"]["reinforcement_learning"]["agent"] = os.path.join(output_dir, "results/Agent.ckpt")

    #     root_output_dir = os.path.expanduser("{}_seed{}".format(jobid, seed))

    #     # Define new directory for the next round
    #     output_dir = os.path.join(root_output_dir, "HITL_iteration{}_{}".format(REINVENT_round, acquisition))
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
    #     print(output_dir)

    #     # modify log and result paths in configuration
    #     configuration["logging"]["logging_path"] = os.path.join(output_dir, "progress.log")
    #     configuration["logging"]["result_folder"] = os.path.join(output_dir, "results")

    #     # write the updated configuration file to the disc
    #     configuration_JSON_path = os.path.join(output_dir, conf_filename)
    #     with open(configuration_JSON_path, 'w') as f:
    #         json.dump(configuration, f, indent=4, sort_keys=True)

    # r = np.arange(len(expert_score))
    # m_score = [np.mean(expert_score[i]) for i in r]
    # print("Mean expert score : ", m_score)

In [69]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [76]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps
batch_size = 80 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "random" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.1 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 50 # number of top molecules to choose from scaffold. 

output_dir = f"score_regression_R{num_rounds}_T{num_iters}_Q{num_queries}_{acquisition}_{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring
Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42
Results will be saved at score_regression_R2_T3_Q10_random_0.1
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: score_regression_R2_T3_Q10_random_0.1/REINVENT_round_1/config.json.
Run REINVENT


Found local copy...


Exit code: 0
Number of SMILES in scaffold_memory.csv:  79
HITL iteration = 1
Loading scoring feedback model.
Loading Score Regression model successfully
Feedback idx at HITL iteration 1: [ 4 32  3 28 19 44 33 49  9 39]
Human score at HITL iteration 1: [0.36197765639556934, 0.018668260291653804, 0.15745703517860277, 0.17864798033269413, 0.23860682283868223, 0.0, 0.18676499739872188, 0.07684694028250622, 0.05398697840056103, 0.05698414426946282]
New queried dataset size: 10
Combined dataset size: 34
Saved augmented training data at score_regression_R2_T3_Q10_random_0.1/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv


TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

### Running Bradley Terry model

In [31]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round

REINVENT_n_steps = 1 # number of REINVENT optimization steps
batch_size = 32 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "thompson" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.1 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
choose_top_smiles = 50 # number of top molecules to choose from scaffold. 

output_dir = f"bradley_terry_R{num_rounds}_T{num_iters}_Q{num_queries}_{acquisition}_{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring
Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42. 
 Results will be saved at output_feedback_comparing
REINVENT round = 1
Creating config file: output_feedback_comparing/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Loading comparing feedback model.
Loading Bradley Terry model successfully


### Running Rank ListNet model

In [53]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking
output_dir = f"output_feedback_{feedback_type}"

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 4 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps

# WARNING: CHOOSING LARGER BATCH SIZE WOULD EXPONENTIALLY INCREASE THE NUMBER OF COMBINATIONS
# BETTER KEEP IT AT 32 OR 64
batch_size = 80 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "thompson" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.0 # noise level for simulated chemist's responses

num_queries = 5 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
choose_top_smiles = 50 # number of top molecules to choose from scaffold. 
        
assert num_queries * num_iters < choose_top_smiles, "num_queries * num_iters must be less than choose_top_smiles"
assert choose_top_smiles < batch_size, "choose_top_smiles must be less than batch_size"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top molecules to choose from scaffold
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring
Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42. 
 Results will be saved at output_feedback_ranking
REINVENT round = 1
Creating config file: output_feedback_ranking/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Loading ranking feedback model.
Loading Rank ListNet model successfully
